## Multi Step Tool Use
- Multi step tool use happens when more than one tool is required and the output of one tool is needed as the input to another tool.
- In other words, tool calling needs to happen in a sequence.
- This is useful in scenarios where tasks have to be broken down into a sequence of steps.
- https://cohere.com/llmu/multi-step-tool-use-2?ref=cohere-ai.ghost.io

### ReAct-Style Prompting
- A common approach for multi-step tool use implementation is using the ReAct style of prompting
- The ReAct framework is a technique for improving the accuracy and reliability of responses from LLMs.
- It combines "Reasoning" and "Acting", inspired by the natural human ability to learn and make decisions by blending these two elements.
- It allows LLMs to create a plan to solve a task and then execute the plan step by step in an iterative process, where it reflects and adjusts its plan after each action it takes.
- By integrating both reasoning and acting within LLMs, ReAct guides them to produce verbal reasoning paths and specific actions for a given task.
- The framework follows a series of Thought-Action-Observation reasoning sequences.
- Given a user input, the LLM first generates a thought, which includes the plan of what it needs to do. Then it generates an action, which is a call to an external tool.
- The result of this tool is then passed to its observation step.
- It will repeat this sequence until a task has been fulfilled.

### Multi-Step Tool use Workflow
The Multi-Step tool workflow uses a style similar to the ReAct framework. The prompt is comprised of:
- The preamble, consisting of the system and custom (if defined) preambles
- The list of available tools, taken from the tool schema defined by the developer
- The user message
- And finally, the instructions to run the ReAct-style workflow.

#### Planning and Execution
- The first instruction to the LLM is to perform a "planning" step consisting of an initial high-level plan of how the model will solve the problem, including the tools and steps required.
- This is followed by actually carrying out the plan by repeatedly using actions (tool calls), with every step going through an Action-Observation-Reflection sequence.

#### Self-Correction
- So far we have assumed that every tool call will be executed successfully and return some result.
- But calling external functions could return errors, such as missing information, unsuccessful attempts and many others.
- In this situation, the model will be able to reflect on this returned result and take a different course of action than what was originally planned. For ex: it might rerun the step that returned error rather than moving to the next step.

### Setting Up the Tools


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv('COHERE_API_KEY', '')

In [2]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import json
import cohere

c:\Users\heena\Desktop\Genetative_AI\genai_env\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
co = cohere.Client(api_key=cohere_api_key)

### Step 1. Create Tools
Let’s now create two tools:

- A function called `list_calendar_events` to list the existing calendar events based on a given date. 
    - For simplicity, we’ll include a mock events record, which is a simple list of events in a day.
- A function called `create_calendar_event` to create a new calendar event based on the provided date, time, and duration. 
    - To keep things simple, we will not make actual changes to a database but instead will just return a simple success message for illustration purposes.

In [5]:
def list_calendar_events(date: str):
  events = [{"start": "8:00", "end": "8:59"}, {"start": "9:00", "end": "9:59"},
            {"start": "11:00", "end": "11:59"},{"start": "12:00", "end": "12:59"}]

  return {
        "existing_events": events
    }

def create_calendar_event(date: str, time: str, duration: int):
  
  return {
        "is_success": True,
        "message": f"Created a {duration} hour long event at {time} on {date}"
    }

functions_map = {
    "list_calendar_events": list_calendar_events,
    "create_calendar_event": create_calendar_event
}

### Step 2: Define Tool Schema

In [6]:
tools = [
    {
      "name": "list_calendar_events",
      "description": "Returns a list of existing events for the specified date, including the start time and end time for each event.",
      "parameter_definitions": {
        "date": {
          "description": "the date to list events for, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        }
      }
    }, 
    {
      "name": "create_calendar_event",
      "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
      "parameter_definitions": {
        "date": {
          "description": "the date on which the event starts, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        },
        "time": {
          "description": "the time of the event, formatted using 24h military time formatting",
          "type": "str",
          "required": True
        },
        "duration": {
          "description": "the number of hours the event lasts for",
          "type": "float",
          "required": True
        }
      }
    }
]

### Step 3: Create Custom Preamble (Optional)

In [7]:
preamble="""## Task & Context
You are a calendar assistant who helps people schedule events on their calendar. You must make sure that a new event does not overlap with any existing event.
Today is Thursday, May 23, 2024
"""

### Step 4: Run the Assistant
- The `run_assistant` function will be the same as in the Single step parallel tool use, where we will loop over multiple tool calls.
- The only difference is that we will remove the `force_single_step` argument as it is `False` by default.
- This tells the API to enable a multi-step workflow and use the right prompt template to run the workflow.

In [8]:
model = "command-r-plus"

def run_assistant(message, chat_history=[]):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Update the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

In [9]:
# Ask the assistant a question
chat_history = run_assistant("How many meetings do I have today")

Question:
How many meetings do I have today
Tool plan:
I will list the events for today and count how many there are. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/24'}
Final response:
You have four meetings today.
Citations:
start=9 end=28 text='four meetings today' document_ids=['list_calendar_events:0:2:0']

Cited Documents:
{'existing_events': '[{"end":"8:59","start":"8:00"},{"end":"9:59","start":"9:00"},{"end":"11:59","start":"11:00"},{"end":"12:59","start":"12:00"}]', 'id': 'list_calendar_events:0:2:0', 'tool_name': 'list_calendar_events'}


In [10]:
# Let’s now try to ask a question that requires multi-step tool calling — 
# this one asking the assistant to book an appointment. To complete this task,
# the assistant will have to first query the existing events and then use the 
# information to create a new event that wouldn’t cause a conflict.
chat_history = run_assistant("Create an hour-long appointment for the first available free slot after 9am")

Question:
Create an hour-long appointment for the first available free slot after 9am
Tool plan:
I will check the user's calendar for 23/05/2024 to find the first available free slot after 9am, then create an hour-long appointment in that slot. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Tool plan:
The user's calendar shows that they have a free slot after 9am from 10:00 to 11:00. I will now create an hour-long appointment in that slot. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '10:00'}
Final response:
I've created an hour-long appointment for you at 10:00.
Citations:
start=16 end=37 text='hour-long appointment' document_ids=['create_calendar_event:0:4:0']
start=49 end=54 text='10:00' document_ids=['create_calendar_event:0:4:0']

Cited Documents:
{'id': 'create_calendar_event:0:4:0', 'is_success': 'true', 'message': 'Created a 1 hour long event at 10:00 on 05/23/2024', 'tool_name': '

In [11]:
chat_history

[Message_User(message='Create an hour-long appointment for the first available free slot after 9am', tool_calls=None, role='USER'),
 Message_Chatbot(message="I will check the user's calendar for 23/05/2024 to find the first available free slot after 9am, then create an hour-long appointment in that slot.", tool_calls=[ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'})], role='CHATBOT'),
 Message_Tool(tool_results=[ToolResult(call=ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'}), outputs=[{'existing_events': [{'end': '8:59', 'start': '8:00'}, {'end': '9:59', 'start': '9:00'}, {'end': '11:59', 'start': '11:00'}, {'end': '12:59', 'start': '12:00'}]}])], role='TOOL'),
 Message_Chatbot(message="The user's calendar shows that they have a free slot after 9am from 10:00 to 11:00. I will now create an hour-long appointment in that slot.", tool_calls=[ToolCall(name='create_calendar_event', parameters={'date': '05/23/2024', 'duration': 1, 'time': '10:

### Multi-Step, Parallel Tool Use

In [12]:
# We will ask the assistant to create two separate events on the same day.
chat_history = run_assistant("Book two hour-long appointments for any available time between 8am to 6pm")

Question:
Book two hour-long appointments for any available time between 8am to 6pm
Tool plan:
I will list the user's calendar events for today, then create two one-hour events at available times. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Tool plan:
The user's calendar is free from 13:00 to 18:00 today. I will create two one-hour events at available times during this period. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '13:00'}
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '14:00'}
Final response:
I've booked two one-hour appointments for you today, at 13:00 and 14:00.
Citations:
start=56 end=61 text='13:00' document_ids=['create_calendar_event:0:4:0']
start=66 end=71 text='14:00' document_ids=['create_calendar_event:1:4:0']

Cited Documents:
{'id': 'create_calendar_event:0:4:0', 'is_success': 'true', 'message': 'Created a 1 hour long eve